### Initilise gym enviroment

In [1]:
import gym
import numpy as np

env = gym.make('gym_connect4:connect4-v0')

action_space_size = env.action_space.n
state_space_size = env.observation_space.n

### Create neural network for action-value approximation

In [45]:
import torch.nn as nn

class DQNAgent():
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.gamma = 0.95    # discount rate
        self.epsilon = 1.0   # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self._build_model()

    def _build_model(self):
        # Neural Net for Deep-Q learning Model
        return nn.Sequential(
                nn.Linear(self.state_size, 24),
                nn.ReLU(),
                nn.Linear(24, 24),
                nn.ReLU(),
                nn.Linear(24, self.action_size))

    def act(self, state):
        # Explore
        if np.random.rand() <= self.epsilon:
            return np.random.choice(action_space_size)
            
        # Greedy
        act_values = self.model.forward(state)
        return np.argmax(act_values)
    
    def learn(self):
        

In [55]:
agent = DQNAgent(42, 7)
agent.act([0]*42)

5

### Train the agent

In [23]:
agent = DQNAgent(state_space_size, action_space_size)

for episode in range(20000):
    state = env.reset()
    episode_rewards = 0
    for t in range(100):
            
        new_state, reward, done, info = env.step(action)
        
        # Update Q-able with oponents move as the resulting state
        q_table[state, action] = q_table[state, action] * (1 - learning_rate) + \
                learning_rate * (reward + discount_rate * np.max(q_table[new_state, :]))
        
        state = new_state
        episode_rewards += reward
        
        # Exploration rate decay
        exploration_rate = min_exploration_rate + \
        (max_exploration_rate - min_exploration_rate) * np.exp(-exploration_decay_rate*episode)
        
        if done:
            print("Episode finished after {} timesteps. Player {} wins".format(t+1, env.game.winner))
            break
env.close()

NameError: name 'Sequential' is not defined

In [16]:
class Agent:
    def __init__()
    

0.653277939214514

In [4]:
x = [1, 1, 1, 0, 2, 4, 2, 2]
from itertools import groupby
for k, g in groupby(x):
    print(k, g)

1 <itertools._grouper object at 0x7f30d470fa90>
0 <itertools._grouper object at 0x7f30e46bd710>
2 <itertools._grouper object at 0x7f30e46bdb90>
4 <itertools._grouper object at 0x7f30d4705a10>
2 <itertools._grouper object at 0x7f30d4705650>


In [19]:
x = {1:[1,2, 4], 2:[5,6 ,6], 3:[4]}
[v for col in x.values() for v in col]

[1, 2, 4, 5, 6, 6, 4]

In [118]:
class connect4:
    def __init__(self):
        self.board = {1: [0]*6,
                      2: [0]*6,
                      3: [0]*6,
                      4: [0]*6,
                      5: [0]*6,
                      6: [0]*6,
                      7: [0]*6}
        self.possible_moves = [i for i in range(1, 8)]
        self.player = 0
        self.winner = None
        
    def make_move(self, move):
        if move not in self.possible_moves:
            raise ValueError("move is not possible")
            
        # Switch the player and play their move in the top of the specified column
        self.player = 1 if self.player != 1 else 2
        top_of_col = self.board[move].index(0)
        self.board[move][top_of_col] = self.player
        
        # Players have reached the top of column so remove it from possible actions
        if top_of_col >= 5:
            self.possible_moves.remove(move)
            
        if self.check_winner(move, top_of_col):
            self.winner = self.player
            
    def check_winner(self, col, row):
        # just to keep variable names short
        val = self.player
        brd = self.board

        # check right 
        if col <= 4:
            # Horizontal
            if brd[col+1][row] == brd[col+2][row] == brd[col+3][row] == val:
                return True
            # Diagonal Down
            if row >= 3 and brd[col+1][row-1] == brd[col+2][row-2] == brd[col+3][row-3] == val:
                return True
            # Diagonal Up
            if row <= 2 and brd[col+1][row+1] == brd[col+2][row+2] == brd[col+3][row+3] == val:
                return True
        
        # check left
        if col >= 4:
            # Horizontal
            if brd[col-1][row] == brd[col-2][row] == brd[col-3][row] == val:
                return True
            # Diagonal Down
            if row >= 3 and brd[col-1][row-1] == brd[col-2][row-2] == brd[col-3][row-3] == val:
                return True
            # Diagonal Up
            if row <= 2 and brd[col-1][row+1] == brd[col-2][row+2] == brd[col-3][row+3] == val:
                return True
            
        # check vertical
        return any(1 for key, group in groupby(brd[col]) if len(list(group)) > 3 and key == val)

In [146]:
game = connect4()
print(game.winner)
game.make_move(2)
print(game.winner)
game.make_move(2)
print(game.winner)
game.make_move(2)
print(game.winner)
game.make_move(2)
game.make_move(2)
game.make_move(2)
print(game.winner)
game.make_move(3)
game.make_move(7)
game.make_move(4)
game.make_move(7)
game.make_move(5)

for x in range(5, -1, -1):
    print([game.board[k][x] for k in game.board])

None
None
None
None
None
[0, 2, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0]
[0, 2, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0]
[0, 2, 0, 0, 0, 0, 2]
[0, 1, 1, 1, 1, 0, 2]


In [126]:
class TicTacToe:
    """
    Board is indexed:
        [0, 1, 2,
         3, 4, 5,
         6, 7, 8]
         
         and contains either 0 or the player number (1 or 2)
         
    player represents the player who last moved
    """
    
    def __init__(self):
        self.board = [0 for i in range(9)]
        self.possible_moves = [i for i in range(9)]
        self.player = 0
    
    def make_move(self, move):
        assert move in self.possible_moves
        
        self.player = 1 if self.player != 1 else 2
        self.board[move] = self.player
        self.possible_moves.remove(move)
        
    def winner(self):
        if all(self.board):
            return 0
        
        lines = ((0,1,2),
                 (3,4,5),
                 (6,7,8),
                 (0,3,6),
                 (1,4,7),
                 (2,5,8),
                 (0,4,8),
                 (2,4,6))
        
        for i, j, k in lines:
            player = self.board[i]
            if player and player == self.board[j] == self.board[k]:
                return player
            
        return -1
    
    def render(self):
        print(self.board[:3])
        print(self.board[3:6])
        print(self.board[6:9])

In [127]:
x = TicTacToe()

In [128]:
while x.winner() is -1:
    x.render()
    x.make_move(int(input("Enter a move: ")))
    
print("\n\nand the winner is: {}".format(x.winner()))

[0, 0, 0]
[0, 0, 0]
[0, 0, 0]
Enter a move: 3
[0, 0, 0]
[1, 0, 0]
[0, 0, 0]
Enter a move: 3


AssertionError: 